In [169]:
from pynwb import NWBHDF5IO
#from nwbwidgets import nwb2widget
import h5py, glob, os
import numpy as np
import matplotlib.pyplot as plt
from dlab import sglx_analysis as sglx
from dlab import psth_and_raster as pr
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm as tqdm
import glob
import pickle as pkl
import json
import datetime as dt
import h5py
from dlab import sglx_analysis as sglx
import math
# import jlh_ephys_tools as jlh
import matplotlib.pyplot as plt
# from open_ephys.analysis import Session
import seaborn as sns
import dlab.psth_and_raster as psth
from dlab.generalephys import cleanAxes
import matplotlib.lines as mlines


# Back up function

In [170]:
def allUnits_rasters_flash(self,probeLetter,path,Savefig=False,):
        df_stim = self.trials()
        df_units = self.units() 

     
        df1 = df_units[df_units.probe==probeLetter][df_units.label==2]
        unit_index = df1.index.tolist()

        # unit = unit_index[unit_indexNumber]
        for unit in unit_index:

            f,ax=plt.subplots(1,1)
            f.suptitle(f"probe {probeLetter} unit {unit} flash epoch" , color='black')
            
                # Set the background color and text color for this subplot
            ax.set_facecolor('black')
            ax.tick_params(color='black', labelcolor='black')
            for spine in ax.spines.values():
                spine.set_edgecolor('black')
            
            
            psth.psth_line(times=df1[df1.index==unit].spike_times.values[0],
                    triggers=df_stim[(df_stim.stimulus=='luminance_flash') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                        ymax=60,binsize=0.05,axes=ax,color='#487697')
            psth.psth_line(times=df1[df1.index==unit].spike_times.values[0],
                    triggers=df_stim[(df_stim.stimulus=='luminance_flash_opto') ].start_time.values,
                        ymax=60,binsize=0.05,axes=ax,color='#ffaa00')
            plt.tight_layout()
            # if Savefig == True:
            #     f.savefig(path+f'{unit}')
            #     f.savefig(path+f'{unit}')
            # else:
            #     pass

            f,ax=plt.subplots(2,1)
                # Set the background color and text color for these subplots
            f.suptitle(f"probe {probeLetter} unit {unit} flash epoch" , color='black')
            for ax_ in ax:
                ax_.set_facecolor('black')
                ax_.tick_params(color='black', labelcolor='black')
                for spine in ax_.spines.values():
                    spine.set_edgecolor('black')
            
            psth.raster(times=df1[df1.index==unit].spike_times.values[0],
                    triggers=df_stim[(df_stim.stimulus=='luminance_flash') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                        axes=ax[0],color='#487697',timeDomain=True,post=1.5,ms=8)
            psth.raster(times=df1[df1.index==unit].spike_times.values[0],
                    triggers=df_stim[(df_stim.stimulus=='luminance_flash_opto') ].start_time.values,
                        axes=ax[1],color='#ffaa00',timeDomain=True,post=1.5,ms=8)
            for ax_ in ax: ax_.set_xlim(-0.5,1.0)
            
            if Savefig == True:
                f.savefig(path+f'{unit}')
                f.savefig(path+f'{unit}')
            else:
                pass
            # if Savefig == True:
            #     plt.tight_layout()
            #     f.savefig(path+f'{unit}')
            # else:
            #     pass

def allUnit_scenes(self, probeLetter, brain_region ,path ,Savefig=False):
    df_stim = self.trials()
    df_units = self.units()

    df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
    unit_index = df1.index.tolist()

    for unit in unit_index:
        # Create a single figure with 3 subplots
        # f, ax = plt.subplots(3, 1)
        f, ax = plt.subplots(3, 1, figsize=(10, 15))

        f.suptitle(f"aligned grating events; unit {unit}, probe {probeLetter} in {brain_region}", color='black')

        # Plot the psth_line in the first subplot
        ax[0].set_facecolor('black')
        ax[0].tick_params(color='black', labelcolor='black')
        for spine in ax[0].spines.values():
            spine.set_edgecolor('black')
        psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                    triggers=df_stim[(df_stim.stimulus == 'scenes') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                    ymax=60, binsize=0.05, axes=ax[0], color='#ffaa00')
        
        psth.psth_line(times=df1[df1.index==unit].spike_times.values[0],
                    triggers=df_stim[(df_stim.stimulus=='scenes_opto') ].start_time.values,
                    ymax=60,binsize=0.05,axes=ax[0],color='#487697')
        ax[0].set_xlim(-0.5, 1.0)
        
        # Create custom legend handles
        handle1 = mlines.Line2D([], [], color='#ffaa00', label='Non-Opto')
        handle2 = mlines.Line2D([], [], color='#487697', label='Opto')

        # Add legend to the first subplot
        ax[0].legend(handles=[handle1, handle2], loc='upper right')


        # Plot the rasters in the second and third subplots
        for i, ax_ in enumerate(ax[1:]):
            ax_.set_facecolor('black')
            ax_.tick_params(color='black', labelcolor='black')
            for spine in ax_.spines.values():
                spine.set_edgecolor('black')

        psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                    triggers=df_stim[(df_stim.stimulus == 'scenes') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                    axes=ax[1], color='#ffaa00', timeDomain=True, post=1.5, ms=8)

        psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                    triggers=df_stim[(df_stim.stimulus == 'scenes_opto')].start_time.values,
                    axes=ax[2], color='#487697', timeDomain=True, post=1.5, ms=8)
        ax[1].set_xlim(-0.5, 1.0)
        ax[2].set_xlim(-0.5, 1.0)
        ax[1].set_title('non-optp', color='black')
        ax[2].set_title('opto', color='black')


        plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the suptitle

        if Savefig:
            f.savefig(f"{path}{unit}.png")
    
        plt.close(f)  # Close the figure to free up memory
        # Uncomment the next line to stop the loop after one iteration
        

def allUnit_scenes_conjoined(self, probeLetter, brain_region, path, Savefig=False):
    df_stim = self.trials()
    df_units = self.units()

    df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
    unit_index = df1.index.tolist()

    # Filter df_stim to only include rows where stimulus is 'scenes' or 'scenes_opto'
    df_stim_filtered = df_stim[df_stim['stimulus'].isin(['scenes', 'scenes_opto'])]
    unique_stimulus_indices = df_stim_filtered['stimulus_index'].unique()

    for unit in unit_index:
        f, ax = plt.subplots(4, 5, figsize=(20, 16))  # 4x5 grid for 20 subplots
        f.suptitle(f"aligned scene events; unit {unit}, probe {probeLetter} in {brain_region}", fontsize=16, color='black')

        ax = ax.flatten()  # Flatten the array for easy indexing

        for i, stim_index in enumerate(unique_stimulus_indices):
            # Set the background color and text color for this subplot
            ax[i].set_facecolor('black')
            ax[i].tick_params(color='black', labelcolor='black')
            for spine in ax[i].spines.values():
                spine.set_edgecolor('black')

            # Filter triggers by stimulus_index
            triggers_non_opto = df_stim_filtered[(df_stim_filtered.stimulus == 'scenes') & (df_stim_filtered.optogenetics_LED_state == 0) & (df_stim_filtered.stimulus_index == stim_index)].start_time.values
            triggers_opto = df_stim_filtered[(df_stim_filtered.stimulus == 'scenes_opto') & (df_stim_filtered.stimulus_index == stim_index)].start_time.values

            psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=triggers_non_opto,
                        ymax=60, binsize=0.05, axes=ax[i], color='#ffaa00')

            psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=triggers_opto,
                        ymax=60, binsize=0.05, axes=ax[i], color='#487697')

            ax[i].set_title(f"Index {stim_index}", color='black')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the suptitle

        if Savefig:
            f.savefig(f"{path}{unit}_all_indices.png")

        plt.close(f)  # Close the figure to free up memory

def allUnit_scenes_conjoined(self, probeLetter, brain_region, path, Savefig=False):
    df_stim = self.trials()
    df_units = self.units()

    df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
    unit_index = df1.index.tolist()

    df_stim_filtered = df_stim[df_stim['stimulus'].isin(['scenes', 'scenes_opto'])]
    unique_stimulus_indices = df_stim_filtered['stimulus_index'].unique()

    for unit in unit_index:
        # First figure for psth_line plots
        f1, ax1 = plt.subplots(4, 5, figsize=(20, 16))
        f1.suptitle(f"aligned scene events (PSTH); unit {unit}, probe {probeLetter} in {brain_region}", fontsize=16, color='black')

        # Second figure for raster plots
        f2, ax2 = plt.subplots(4, 5, figsize=(20, 16))
        f2.suptitle(f"aligned scene events (Raster); unit {unit}, probe {probeLetter} in {brain_region}", fontsize=16, color='black')

        ax1 = ax1.flatten()
        ax2 = ax2.flatten()

        for i, stim_index in enumerate(unique_stimulus_indices):
            for ax in [ax1[i], ax2[i]]:
                ax.set_facecolor('black')
                ax.tick_params(color='black', labelcolor='black')
                for spine in ax.spines.values():
                    spine.set_edgecolor('black')

            triggers_non_opto = df_stim_filtered[(df_stim_filtered.stimulus == 'scenes') & (df_stim_filtered.optogenetics_LED_state == 0) & (df_stim_filtered.stimulus_index == stim_index)].start_time.values
            triggers_opto = df_stim_filtered[(df_stim_filtered.stimulus == 'scenes_opto') & (df_stim_filtered.stimulus_index == stim_index)].start_time.values

            # PSTH plots
            psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=triggers_non_opto,
                        ymax=60, binsize=0.05, axes=ax1[i], color='#ffaa00')
            psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=triggers_opto,
                        ymax=60, binsize=0.05, axes=ax1[i], color='#487697')

            # Raster plots
            psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=triggers_non_opto,
                        axes=ax2[i], color='#ffaa00', timeDomain=True, post=1.5, ms=8)
            psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=triggers_opto,
                        axes=ax2[i], color='#487697', timeDomain=True, post=1.5, ms=8)

            ax1[i].set_title(f"Index {stim_index}", color='black')
            ax2[i].set_title(f"Index {stim_index}", color='black')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])

        if Savefig:
            f1.savefig(f"{path}{unit}_all_indices_psth.png")
            f2.savefig(f"{path}{unit}_all_indices_raster.png")

        plt.close(f1)
        plt.close(f2)

def allUnit_scenes_conjoined(self, probeLetter, brain_region, path, Savefig=False):
    df_stim = self.trials()
    df_units = self.units()

    df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
    unit_index = df1.index.tolist()

    df_stim_filtered = df_stim[df_stim['stimulus'].isin(['scenes', 'scenes_opto'])]
    unique_stimulus_indices = df_stim_filtered['stimulus_index'].unique()

    for unit in unit_index:
        f1, ax1 = plt.subplots(4, 5, figsize=(20, 16))
        f1.suptitle(f"aligned scene events (PSTH); unit {unit}, probe {probeLetter} in {brain_region}", fontsize=16, color='black')

        f2, ax2 = plt.subplots(4, 10, figsize=(40, 16))  # 4 rows, 10 columns for 20 pairs of raster plots
        f2.suptitle(f"aligned scene events (Raster); unit {unit}, probe {probeLetter} in {brain_region}", fontsize=16, color='black')

        ax1 = ax1.flatten()
        ax2 = ax2.reshape(4, 5, 2)  # Reshape to 4x5x2 for easy indexing

        for i, stim_index in enumerate(unique_stimulus_indices):
            ax1[i].set_facecolor('black')
            ax1[i].tick_params(color='black', labelcolor='black')
            for spine in ax1[i].spines.values():
                spine.set_edgecolor('black')

            for ax in ax2[i // 5, i % 5]:
                ax.set_facecolor('black')
                ax.tick_params(color='black', labelcolor='black')
                for spine in ax.spines.values():
                    spine.set_edgecolor('black')

            triggers_non_opto = df_stim_filtered[(df_stim_filtered.stimulus == 'scenes') & (df_stim_filtered.optogenetics_LED_state == 0) & (df_stim_filtered.stimulus_index == stim_index)].start_time.values
            triggers_opto = df_stim_filtered[(df_stim_filtered.stimulus == 'scenes_opto') & (df_stim_filtered.stimulus_index == stim_index)].start_time.values

            # PSTH plots
            psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=triggers_non_opto,
                        ymax=60, binsize=0.05, axes=ax1[i], color='#ffaa00')
            psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=triggers_opto,
                        ymax=60, binsize=0.05, axes=ax1[i], color='#487697')

            # Raster plots (non-opto)
            psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=triggers_non_opto,
                        axes=ax2[i // 5, i % 5, 0], color='#ffaa00', timeDomain=True, post=1.5, ms=8)

            # Raster plots (opto)
            psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=triggers_opto,
                        axes=ax2[i // 5, i % 5, 1], color='#487697', timeDomain=True, post=1.5, ms=8)

            ax1[i].set_title(f"Index {stim_index}", color='black')
            ax2[i // 5, i % 5, 0].set_title(f"Index {stim_index} Non-Opto", color='black')
            ax2[i // 5, i % 5, 1].set_title(f"Index {stim_index} Opto", color='black')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])

        if Savefig:
            f1.savefig(f"{path}{unit}_all_indices_psth.png")
            f2.savefig(f"{path}{unit}_all_indices_raster.png")

        plt.close(f1)
        plt.close(f2)
        break




In [239]:
class Eopn3_Ephys:

    def __init__(self,nwb_path):
       self.nwb_path = nwb_path
       self.load_nwb()
 
      

    def nwb(self):
        if self.nwb is None:
            self.load_nwb()
        return self.nwb

    def load_nwb(self):
        nwb_path = self.nwb_path
        io = NWBHDF5IO(nwb_path, 'r')
        nwb = io.read()
        self.nwb = nwb
        return self.nwb
    
# prints the metaData associated with nwb
    def view_nwb(self):
        nwb = self.nwb
        print(nwb)

# creates a df of the trails associated with nwb. the Data frame contains data about each trail and its structure 
    def trials(self):
        nwb = self.nwb
        df_stim = nwb.trials.to_dataframe()
        # df_stim.loc[2100:2699,'contacts'] = '10r'  #specific to this recording, fixes an error in dataframe
        self.df_stim = df_stim
        return self.df_stim

# creates the Units Data frame associated with nwb. which contains all the data about the sorted spike units from the recording
    def units(self):
        nwb = self.nwb
        df_units = nwb.units.to_dataframe()
        self.df_units = df_units
        return self.df_units
    
# creates the optogenetics_states Data frame associated with nwb. which contains all the data about the optogenetics states 
    def optogenetics_states(self):
        nwb = self.nwb
        optogenetics_states_df = nwb.intervals['optogenetics_states'].to_dataframe()
        self.optogenetics_states_df = optogenetics_states_df
        return optogenetics_states_df
    
# creates the epochs Data frame associated with nwb. which contains just the entire recording length. 
    def epochs(self):
        nwb = self.nwb
        epochs_df = nwb.intervals['epochs'].to_dataframe()
        self.epochs_df = epochs_df
        return epochs_df
    

    def singleUnit_flash(self,probeLetter,unit_indexNumber,Savefig=False,):
        df_stim = self.trials()
        df_units = self.units() 

        df1 = df_units[df_units.probe==probeLetter][df_units.label==2]
        unit_index = df1.index.tolist()

        print(f"neuron indexs for {probeLetter} {list(unit_index)}")
       
        unit = unit_index[unit_indexNumber]
        print(f"looking at neuron/unit {unit}")
        f,ax=plt.subplots(1,1)
         
            # Set the background color and text color for this subplot
        ax.set_facecolor('black')
        ax.tick_params(color='black', labelcolor='black')
        for spine in ax.spines.values():
            spine.set_edgecolor('black')

        psth.psth_line(times=df1[df1.index==unit].spike_times.values[0],
                triggers=df_stim[(df_stim.stimulus=='luminance_flash') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                    ymax=60,binsize=0.05,axes=ax,color='#487697')
        psth.psth_line(times=df1[df1.index==unit].spike_times.values[0],
                triggers=df_stim[(df_stim.stimulus=='luminance_flash_opto') ].start_time.values,
                    ymax=60,binsize=0.05,axes=ax,color='#ffaa00')
        plt.tight_layout()
        if Savefig == True:
            f.savefig('/Users/danieljdenman/Academics/grants/applications/20230305_R01_NEI_resub/figures/eArch_LM_psth.eps')
            f.savefig('/Users/danieljdenman/Academics/grants/applications/20230305_R01_NEI_resub/figures/eArch_LM_psth.png')
        else:
            pass

        f,ax=plt.subplots(2,1)
            # Set the background color and text color for these subplots
        for ax_ in ax:
            ax_.set_facecolor('black')
            ax_.tick_params(color='black', labelcolor='black')
            for spine in ax_.spines.values():
                spine.set_edgecolor('black')
        
        psth.raster(times=df1[df1.index==unit].spike_times.values[0],
                triggers=df_stim[(df_stim.stimulus=='luminance_flash') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                    axes=ax[0],color='#487697',timeDomain=True,post=1.5,ms=8)
        psth.raster(times=df1[df1.index==unit].spike_times.values[0],
                triggers=df_stim[(df_stim.stimulus=='luminance_flash_opto') ].start_time.values,
                    axes=ax[1],color='#ffaa00',timeDomain=True,post=1.5,ms=8)
        for ax_ in ax: ax_.set_xlim(-0.5,1.0)
     
        plt.tight_layout()
        # f.savefig('/Users/danieljdenman/Academics/grants/applications/20230305_R01_NEI_resub/figures/eArch_LM_raster.eps')


# Save raters and psth_line plots for every unit in the selected probeLetter. Supply a unique save path for eachprobe 
    def allUnits_flash(self, probeLetter, brain_region ,path ,Savefig=False):
        df_stim = self.trials()
        df_units = self.units()

        df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
        unit_index = df1.index.tolist()

        for unit in unit_index:
            # Create a single figure with 3 subplots
            # f, ax = plt.subplots(3, 1)
            f, ax = plt.subplots(3, 1, figsize=(10, 15))

            f.suptitle(f"aligned flash events; unit {unit}, probe {probeLetter} in {brain_region}", color='black')

            # Plot the psth_line in the first subplot
            ax[0].set_facecolor('black')
            ax[0].tick_params(color='black', labelcolor='black')
            for spine in ax[0].spines.values():
                spine.set_edgecolor('black')
            psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=df_stim[(df_stim.stimulus == 'luminance_flash') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                        ymax=60, binsize=0.05, axes=ax[0], color='#ffaa00')
            
            psth.psth_line(times=df1[df1.index==unit].spike_times.values[0],
                        triggers=df_stim[(df_stim.stimulus=='luminance_flash_opto') ].start_time.values,
                        ymax=60,binsize=0.05,axes=ax[0],color='#487697')
            ax[0].set_xlim(-0.5, 1.0)
         
            # Create custom legend handles
            handle1 = mlines.Line2D([], [], color='#ffaa00', label='Non-Opto')
            handle2 = mlines.Line2D([], [], color='#487697', label='Opto')

            # Add legend to the first subplot
            ax[0].legend(handles=[handle1, handle2], loc='upper right')
  

            # Plot the rasters in the second and third subplots
            for i, ax_ in enumerate(ax[1:]):
                ax_.set_facecolor('black')
                ax_.tick_params(color='black', labelcolor='black')
                for spine in ax_.spines.values():
                    spine.set_edgecolor('black')

            psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=df_stim[(df_stim.stimulus == 'luminance_flash') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                        axes=ax[1], color='#ffaa00', timeDomain=True, post=1.5, ms=8)

            psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=df_stim[(df_stim.stimulus == 'luminance_flash_opto')].start_time.values,
                        axes=ax[2], color='#487697', timeDomain=True, post=1.5, ms=8)
            ax[1].set_xlim(-0.5, 1.0)
            ax[2].set_xlim(-0.5, 1.0)
            ax[1].set_title('non-optp', color='black')
            ax[2].set_title('opto', color='black')


            plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the suptitle

            if Savefig:
                f.savefig(f"{path}{unit}.png")

            plt.close(f)  # Close the figure to free up memory

    # def allUnits_flash_conjoined(self, probeLetter, brain_region, path, Savefig=False):
    #     df_stim = self.trials()
    #     df_units = self.units()

    #     df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
    #     unit_index = df1.index.tolist()

    #     # Calculate the number of rows and columns for the subplots
    #     n_units = len(unit_index)
    #     n_cols = 4  # You can change this to fit your needs
    #     n_rows = math.ceil(n_units / n_cols)

    #     # Create a single figure with multiple subplots
    #     f, axarr = plt.subplots(n_rows, n_cols, figsize=(15, 15))

    #     f.suptitle(f"Aligned flash events for probe {probeLetter} in {brain_region}", color='black')

    #     for i, unit in enumerate(unit_index):
    #         row = i // n_cols
    #         col = i % n_cols
    #         ax = axarr[row, col]

    #         ax.set_facecolor('black')
    #         ax.tick_params(color='black', labelcolor='black')
    #         for spine in ax.spines.values():
    #             spine.set_edgecolor('black')

    #         psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
    #                     triggers=df_stim[(df_stim.stimulus == 'luminance_flash') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
    #                     ymax=60, binsize=0.05, axes=ax, color='#ffaa00')

    #         psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
    #                     triggers=df_stim[(df_stim.stimulus == 'luminance_flash_opto')].start_time.values,
    #                     ymax=60, binsize=0.05, axes=ax, color='#487697')

    #         ax.set_xlim(-0.5, 1.0)
    #         ax.set_title(f"Unit {unit}", color='black')

    #         # Add a legend to the figure
    #         handle1 = mlines.Line2D([], [], color='#ffaa00', label='Non-Opto')
    #         handle2 = mlines.Line2D([], [], color='#487697', label='Opto')
    #         f.legend(handles=[handle1, handle2], loc='upper right')

    #         plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the suptitle

    #         if Savefig:
    #             f.savefig(f"{path}all_units.png")

    #         plt.close(f)  # Close the figure to free up memory


    def allUnits_flash_conjoined(self, probeLetter, brain_region, path, Savefig=False):
        df_stim = self.trials()
        df_units = self.units()

        df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
        unit_index = df1.index.tolist()

        n_units = len(unit_index)
        n_cols = 4
        n_rows = math.ceil(n_units / n_cols)

        f, axarr = plt.subplots(n_rows, n_cols, figsize=(20, 20))  # Increase figure size for more room

        f.suptitle(f"Aligned flash events for probe {probeLetter} in {brain_region}", color='black')

        for i, unit in enumerate(unit_index):
            row = i // n_cols
            col = i % n_cols
            ax = axarr[row, col]

            ax.set_facecolor('black')
            ax.tick_params(color='black', labelcolor='black',labelsize=4)
            for spine in ax.spines.values():
                spine.set_edgecolor('black')

            psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=df_stim[(df_stim.stimulus == 'luminance_flash') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                        ymax=60, binsize=0.05, axes=ax, color='#ffaa00')

            psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=df_stim[(df_stim.stimulus == 'luminance_flash_opto')].start_time.values,
                        ymax=60, binsize=0.05, axes=ax, color='#487697')

            ax.set_xlim(-0.5, 1.0)
            ax.set_title(f"Unit {unit}", color='black')

            # Add a legend to the figure
            handle1 = mlines.Line2D([], [], color='#ffaa00', label='Non-Opto')
            handle2 = mlines.Line2D([], [], color='#487697', label='Opto')
            f.legend(handles=[handle1, handle2], loc='upper right')

            plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the suptitle

            plt.subplots_adjust(hspace=1.0, wspace=1.0)  # Adjust spacing between subplots
            plt.tight_layout(pad=8, h_pad=8, w_pad=8, rect=[0, 0.03, 1, 0.95])  # Increase padding

            if Savefig:
                f.savefig(f"{path}all_units.png")

        plt.close(f)  # Move this outside the loop


    def allUnit_gratings(self, probeLetter, brain_region ,path ,Savefig=False):
        df_stim = self.trials()
        df_units = self.units()

        df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
        unit_index = df1.index.tolist()

        for unit in unit_index:
            # Create a single figure with 3 subplots
            # f, ax = plt.subplots(3, 1)
            f, ax = plt.subplots(3, 1, figsize=(10, 15))

            f.suptitle(f"aligned grating events; unit {unit}, probe {probeLetter} in {brain_region}", color='black')

            # Plot the psth_line in the first subplot
            ax[0].set_facecolor('black')
            ax[0].tick_params(color='black', labelcolor='black')
            for spine in ax[0].spines.values():
                spine.set_edgecolor('black')
            psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=df_stim[(df_stim.stimulus == 'gratings') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                        ymax=60, binsize=0.05, axes=ax[0], color='#ffaa00')
            
            psth.psth_line(times=df1[df1.index==unit].spike_times.values[0],
                        triggers=df_stim[(df_stim.stimulus=='gratings_opto') ].start_time.values,
                        ymax=60,binsize=0.05,axes=ax[0],color='#487697')
            ax[0].set_xlim(-0.5, 1.0)
         
            # Create custom legend handles
            handle1 = mlines.Line2D([], [], color='#ffaa00', label='Non-Opto')
            handle2 = mlines.Line2D([], [], color='#487697', label='Opto')

            # Add legend to the first subplot
            ax[0].legend(handles=[handle1, handle2], loc='upper right')
  

            # Plot the rasters in the second and third subplots
            for i, ax_ in enumerate(ax[1:]):
                ax_.set_facecolor('black')
                ax_.tick_params(color='black', labelcolor='black')
                for spine in ax_.spines.values():
                    spine.set_edgecolor('black')

            psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=df_stim[(df_stim.stimulus == 'gratings') & (df_stim.optogenetics_LED_state == 0)].start_time.values,
                        axes=ax[1], color='#ffaa00', timeDomain=True, post=1.5, ms=8)

            psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                        triggers=df_stim[(df_stim.stimulus == 'gratings_opto')].start_time.values,
                        axes=ax[2], color='#487697', timeDomain=True, post=1.5, ms=8)
            ax[1].set_xlim(-0.5, 1.0)
            ax[2].set_xlim(-0.5, 1.0)
            ax[1].set_title('non-optp', color='black')
            ax[2].set_title('opto', color='black')


            plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the suptitle

            if Savefig:
                f.savefig(f"{path}{unit}.png")
     
            plt.close(f)  # Close the figure to free up memory
            # Uncomment the next line to stop the loop after one iteration

   



    def allUnit_scenes(self, probeLetter, brain_region, path, Savefig=False):
        df_stim = self.trials()
        df_units = self.units()

        df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
        unit_index = df1.index.tolist()

    
        # Filter df_stim to only include rows where stimulus is 'scenes' or 'scenes_opto'
        df_stim_filtered = df_stim[df_stim['stimulus'].isin(['scenes', 'scenes_opto'])]

        unique_stimulus_indices = df_stim_filtered['stimulus_index'].unique()

        for unit in unit_index:
            for stim_index in unique_stimulus_indices:
                f, ax = plt.subplots(3, 1, figsize=(10, 15))

                f.suptitle(f"aligned scene events; unit {unit}, probe {probeLetter} in {brain_region}, stimulus_index {stim_index}", color='black')

                # Plot the psth_line in the first subplot
                ax[0].set_facecolor('black')
                ax[0].tick_params(color='black', labelcolor='black')
                for spine in ax[0].spines.values():
                    spine.set_edgecolor('black')

                # ... (rest of your code remains the same, but filter triggers by stimulus_index)
                
                triggers_non_opto = df_stim_filtered[(df_stim_filtered.stimulus == 'scenes') & (df_stim_filtered.optogenetics_LED_state == 0) & (df_stim_filtered.stimulus_index == stim_index)].start_time.values
                triggers_opto = df_stim_filtered[(df_stim_filtered.stimulus == 'scenes_opto') & (df_stim_filtered.stimulus_index == stim_index)].start_time.values

                psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                            triggers=triggers_non_opto,
                            ymax=60, binsize=0.05, axes=ax[0], color='#ffaa00')
                
                psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                            triggers=triggers_opto,
                            ymax=60, binsize=0.05, axes=ax[0], color='#487697')
                
                ax[0].set_xlim(-0.5, 1.0)

                    
                # Create custom legend handles
                handle1 = mlines.Line2D([], [], color='#ffaa00', label='Non-Opto')
                handle2 = mlines.Line2D([], [], color='#487697', label='Opto')

                # Add legend to the first subplot
                ax[0].legend(handles=[handle1, handle2], loc='upper right')
    

                   # Plot the rasters in the second and third subplots
                for i, ax_ in enumerate(ax[1:]):
                    ax_.set_facecolor('black')
                    ax_.tick_params(color='black', labelcolor='black')
                    for spine in ax_.spines.values():
                        spine.set_edgecolor('black')
                    
                # ... (rest of your code remains the same, but filter triggers by stimulus_index)

                psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                            triggers=triggers_non_opto,
                            axes=ax[1], color='#ffaa00', timeDomain=True, post=1.5, ms=8)

                psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                            triggers=triggers_opto,
                            axes=ax[2], color='#487697', timeDomain=True, post=1.5, ms=8)
                
                ax[1].set_xlim(-0.5, 1.0)
                ax[2].set_xlim(-0.5, 1.0)
                ax[1].set_title('non-optp', color='black')
                ax[2].set_title('opto', color='black')

                # ... (rest of your code for saving and closing figures)
                plt.tight_layout(rect=[0, 0.03, 1, 0.92])  # Adjust the top value

                if Savefig:
                    f.savefig(f"{path}{unit}_stimulus_index_{stim_index}.png")

                plt.close(f)
 


    def allUnit_scenes_conjoined(self, probeLetter, brain_region, path, Savefig=False):
        df_stim = self.trials()
        df_units = self.units()

        df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
        unit_index = df1.index.tolist()

        df_stim_filtered = df_stim[df_stim['stimulus'].isin(['scenes', 'scenes_opto'])]
        unique_stimulus_indices = df_stim_filtered['stimulus_index'].unique()

        for unit in unit_index:
            f1, ax1 = plt.subplots(4, 5, figsize=(25, 30))  # Increase the size

            f1.suptitle(f"aligned scene events (PSTH); unit {unit}, probe {probeLetter} in {brain_region}", fontsize=16, color='black')

            f2, ax2 = plt.subplots(10, 4, figsize=(20, 32))  # 8 rows, 5 columns
            f2.suptitle(f"aligned scene events (Raster); unit {unit}, probe {probeLetter} in {brain_region}", fontsize=16, color='black')

            ax1 = ax1.flatten()
            ax2 = ax2.reshape(-1, 2)  # Reshape to 20 pairs of 2 subplots

            for i, stim_index in enumerate(unique_stimulus_indices):
                ax1[i].set_facecolor('black')
                ax1[i].tick_params(color='black', labelcolor='black')
                for spine in ax1[i].spines.values():
                    spine.set_edgecolor('black')

                for ax in ax2[i]:
                    ax.set_facecolor('black')
                    ax.tick_params(color='black', labelcolor='black')
                    for spine in ax.spines.values():
                        spine.set_edgecolor('black')

                triggers_non_opto = df_stim_filtered[(df_stim_filtered.stimulus == 'scenes') & (df_stim_filtered.optogenetics_LED_state == 0) & (df_stim_filtered.stimulus_index == stim_index)].start_time.values
                triggers_opto = df_stim_filtered[(df_stim_filtered.stimulus == 'scenes_opto') & (df_stim_filtered.stimulus_index == stim_index)].start_time.values

                # PSTH plots
                psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                                triggers=triggers_non_opto,
                                ymax=60, binsize=0.05, axes=ax1[i], color='#ffaa00')
                psth.psth_line(times=df1[df1.index == unit].spike_times.values[0],
                                triggers=triggers_opto,
                                ymax=60, binsize=0.05, axes=ax1[i], color='#487697')

                # Raster plots (non-opto)
                psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                            triggers=triggers_non_opto,
                            axes=ax2[i, 0], color='#ffaa00', timeDomain=True, post=1.5, ms=8)

                # Raster plots (opto)
                psth.raster(times=df1[df1.index == unit].spike_times.values[0],
                            triggers=triggers_opto,
                            axes=ax2[i, 1], color='#487697', timeDomain=True, post=1.5, ms=8)

                ax1[i].set_title(f"Index {stim_index}", color='black')
                ax2[i, 0].set_title(f"Index {stim_index} Non-Opto", color='black')
                ax2[i, 1].set_title(f"Index {stim_index} Opto", color='black')

                ax1[i].set_xlim(-0.5, 1.0)
                ax2[i, 0].set_xlim(-0.5, 1.0)
                ax2[i, 1].set_xlim(-0.5, 1.0)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])

            if Savefig:
                f1.savefig(f"{path}{unit}_all_indices_psth.png")
                f2.savefig(f"{path}{unit}_all_indices_raster.png")

            plt.close(f1)
            plt.close(f2)
            




In [240]:
nwb_path = r"C:/Users/denmanlab/Desktop/nwb/pa_Bont2_nwb"

paBont2 = Eopn3_Ephys(nwb_path)

In [241]:
# path = r"G:/Grant/rotation/2023-09-29_16-07-39_PaBont2/plots/gratings/probeC/"

# paBont2.allUnit_gratings("C","LGN",path,True)

In [242]:
path = r"G:/Grant/rotation/2023-09-29_16-07-39_PaBont2/plots/flashes/conjoined_plots/probeA/"
paBont2.allUnits_flash_conjoined("A","V1", path,True,)

C:\Users\denmanlab\AppData\Local\Temp\ipykernel_32\2427961651.py:227: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]
C:\Users\denmanlab\AppData\Local\Temp\ipykernel_32\2427961651.py:264: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations.
  plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the suptitle
C:\Users\denmanlab\AppData\Local\Temp\ipykernel_32\2427961651.py:267: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations.
  plt.tight_layout(pad=8, h_pad=8, w_pad=8, rect=[0, 0.03, 1, 0.95])  # Increase padding


In [ ]:
path = r"G:/Grant/rotation/2023-09-29_16-07-39_PaBont2/plots/scenes/conjoined_plots/probeB/"

paBont2.allUnit_scenes_conjoined("B","LM",path,True)

C:\Users\denmanlab\AppData\Local\Temp\ipykernel_32\423317762.py:377: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]


In [ ]:
# path = r"G:/Grant/rotation/2023-09-29_16-07-39_PaBont2/plots/scenes/probeA/"

# paBont2.allUnit_scenes("A","V1",path,True)

C:\Users\denmanlab\AppData\Local\Temp\ipykernel_32\3612892158.py:306: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df_units[df_units.probe == probeLetter][df_units.label == 2]


In [21]:
df = paBont2.trials()
filtered_df = df[df['stimulus'] == 'gratings']
filtered_df[10:50]


# Flash 
## Save all units algined raster plot 
### Change path and probe number 

In [6]:
# path = r"G:/Grant/rotation/2023-09-29_16-07-39_PaBont2/plots/flashes/probeC/"
# paBont2.allUnits_flash("C","LGN", path,True,)

# nwb dataframes

In [196]:
paBont2.trials()

,start_time,stop_time,stimulus,optogenetics_LED_state,flash_luminance,contrast,temporal_frequency,spatial_frequency,orientation,stimulus_index
id,,,,,,,,,,
0,3570.826467,3573.829202,luminance_flash,0,0,-1,-1,-1,-1,-1
1,3573.820467,3576.823202,luminance_flash,0,1,-1,-1,-1,-1,-1
2,3576.823233,3579.825968,luminance_flash,0,0,-1,-1,-1,-1,-1
3,3579.825867,3582.828602,luminance_flash,0,1,-1,-1,-1,-1,-1
4,3582.828467,3585.831202,luminance_flash,0,0,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
27475,10118.996425,10119.997369,scenes_opto,1,-1,-1,-1,-1,-1,15
27476,10119.997359,10120.998302,scenes_opto,1,-1,-1,-1,-1,-1,16
27477,10120.998259,10121.999202,scenes_opto,1,-1,-1,-1,-1,-1,17


In [149]:
paBont2.units()


,depth,xpos,ypos,label,KSlabel,KSamplitude,KScontamination,probe,spike_times
id,,,,,,,,,
0,921.653447,25.536051,3711.653447,0,1,495.0,39.7,A,"[3386.173221728945, 3386.239688022155, 3386.25..."
1,969.001978,26.680260,3759.001978,0,1,827.5,0.0,A,"[4455.821774831464, 4804.668877247732, 4998.07..."
2,881.196474,28.819949,3671.196474,0,1,840.0,59.0,A,"[5196.214778007007, 5221.6585604913225, 5287.8..."
3,900.895605,28.271477,3690.895605,0,1,594.9,0.0,A,"[5810.06179809548, 6378.027536186321, 6384.207..."
4,955.926077,14.131495,3745.926077,0,1,810.6,47.0,A,"[4980.273137009042, 5093.840808931818, 5267.58..."
...,...,...,...,...,...,...,...,...,...
2164,415.393089,47.161546,805.393089,2,2,1747.9,0.0,C,"[3500.652429949263, 3977.238294823803, 4120.03..."
2165,248.302026,23.623893,638.302026,2,2,3077.9,3.1,C,"[3606.4776747480346, 3725.737318863497, 3859.0..."
2166,153.365269,47.669656,543.365269,2,2,857.6,2.8,C,"[3408.0808037098323, 3501.647791848024, 3523.2..."


In [12]:
paBont2.epochs()

,start_time,stop_time,tags
id,,,
0,3570.826467,3871.097233,[luminance_flash]
1,3877.891667,4481.407900,[spatioluminance_noise]
2,4486.933367,5769.590833,[gratings]
3,5782.689433,6785.631842,[scenes]
4,6820.320900,7120.597800,[luminance_flash_opto]
5,7120.976300,7761.235467,[spatioluminance_noise_opto]
6,7796.989433,9079.630758,[gratings_opto]
7,9123.057627,10126.000259,[scenes_opto]


In [13]:
paBont2.optogenetics_states()

,start_time,stop_time,power,wavelength
id,,,,
0,3570.420033,3570.420133,1.5,565
1,6785.688467,6786.702567,1.5,565
2,6787.725200,6788.741167,1.5,565
3,6789.758033,6790.778500,1.5,565
4,6791.794433,6792.796300,1.5,565
...,...,...,...,...
72,10145.145060,10146.161494,1.5,565
73,10147.162494,10148.163761,1.5,565
74,10149.168361,10150.181461,1.5,565
